# Performing raster and vector operations in Python using pyjeo

**Material provided by Dr. Pieter Kempeneers**

[pyjeo](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/) is the follow up of [PKTOOLS](http://pktools.nongnu.org/html/index.html), a suite of utilities written in C++ for image processing with a focus on remote sensing applications. It is distributed under a General Public License (GPLv3) as a Python package.

The examples in this notebook replicate the exercises on [pktools](http://spatial-ecology.net/docs/build/html/PKTOOLS/pktools_osgeo.html) in order to appreciate the difference and still analogy for those that are familiar with pktools.

In a nutshell, the main differences between pyjeo and pktools from a user's perspective are:

- pyjeo is a Python package should be run in a **Python environment**, whereas pktools applications are run from the command line (e.g., in a bash shell)
- pyjeo runs with images entirely **in memory**, whereas pktools runs most applications line per line. This makes pyjeo considerably faster, but with a larger memory footprint. However, there are some methods implemented in pyjeo to reduce the memory footprint by tiling the image

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'numpy'

## Creating masks

**pktools**
We create three masks using different threshold values with [pkgetmask](http://pktools.nongnu.org/html/md_pkgetmask.html)

In [2]:
%%bash
pkgetmask  -co COMPRESS=DEFLATE -co ZLEVEL=9 -min  1  -max  2 -data 1 -nodata 0 -ot Byte  -i  geodata/vegetation/ETmean08-11.tif  -o geodata/vegetation/ETmean08-11_01_trhA.tif
pkgetmask  -co COMPRESS=DEFLATE -co ZLEVEL=9 -min  5  -max  8 -data 1 -nodata 0 -ot Byte  -i  geodata/vegetation/ETmean08-11.tif  -o geodata/vegetation/ETmean08-11_01_trhB.tif
pkgetmask  -co COMPRESS=DEFLATE -co ZLEVEL=9 -min  0  -max  10 -data 0 -nodata 1 -ot Byte  -i  geodata/vegetation/ETmean08-11.tif  -o geodata/vegetation/ETmean08-11_01_trhC.tif

0

terminate called after throwing an instance of '

std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >'


bash: line 1: 164531 Aborted                 (core dumped) pkgetmask -co COMPRESS=DEFLATE -co ZLEVEL

=9 -min 1 -max 2 -data 1 -nodata 0 -ot Byte -i geodata/vegetation/ETmean08-11.tif -o geodata/vegetat

...

ar>, std::allocator<char> >'


bash: line 3: 164593 Aborted                 (core dumped) pkgetmask -co COMPRESS=DEFLATE -co ZLEVEL

=9 -min 0 -max 10 -data 0 -nodata 1 -ot Byte -i geodata/vegetation/ETmean08-11.tif -o geodata/vegeta

tion/ETmean08-11_01_trhC.tif


CalledProcessError: Command 'b'pkgetmask  -co COMPRESS=DEFLATE -co ZLEVEL=9 -min  1  -max  2 -data 1 -nodata 0 -ot Byte  -i  geodata/vegetation/ETmean08-11.tif  -o geodata/vegetation/ETmean08-11_01_trhA.tif\npkgetmask  -co COMPRESS=DEFLATE -co ZLEVEL=9 -min  5  -max  8 -data 1 -nodata 0 -ot Byte  -i  geodata/vegetation/ETmean08-11.tif  -o geodata/vegetation/ETmean08-11_01_trhB.tif\npkgetmask  -co COMPRESS=DEFLATE -co ZLEVEL=9 -min  0  -max  10 -data 0 -nodata 1 -ot Byte  -i  geodata/vegetation/ETmean08-11.tif  -o geodata/vegetation/ETmean08-11_01_trhC.tif\n'' returned non-zero exit status 134.

**pyjeo**

With pyjeo we create the masks in memory in a "pythonic" way using [get items](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html#get-jim-items) without the need to write temporary files.

In [3]:
import pyjeo as pj

ModuleNotFoundError: No module named 'pyjeo'

In [4]:
fn = Path('geodata/vegetation/ETmean08-11.tif')
jim = pj.Jim(fn)

#get mask
mask1 = (jim>=1) & (jim<=2)
mask2 = (jim>=5) & (jim<=8)
mask3 = (jim<0) | (jim>10)

NameError: name 'pj' is not defined

## Applying masks

**pktools** 

Use the prepared mask and apply to the image with [pksetmask](http://pktools.nongnu.org/html/md_pksetmask.html)

In [5]:
%%bash
pksetmask -co COMPRESS=DEFLATE -co ZLEVEL=9 \
-m geodata/vegetation/ETmean08-11_01_trhA.tif  -msknodata 1 -nodata  -9 \
-m geodata/vegetation/ETmean08-11_01_trhB.tif  -msknodata 1 -nodata  -5 \
-m geodata/vegetation/ETmean08-11_01_trhC.tif  -msknodata 1 -nodata -10 \
-i geodata/vegetation/ETmean08-11.tif -o geodata/vegetation/ETmean08-11_01_msk.tif

terminate called after throwing an instance of 'std::__cxx11::basic_string<char, std::char_traits<ch

ar>, std::allocator<char> >'


bash: line 5: 164609 Aborted                 (core dumped) pksetmask -co COMPRESS=DEFLATE -co ZLEVEL

=9 -m geodata/vegetation/ETmean08-11_01_trhA.tif -msknodata 1 -nodata -9 -m geodata/vegetation/ETmea

n08-11_01_trhB.tif -msknodata 1 -nodata -5 -m geodata/vegetation/ETmean08-11_01_trhC.tif -msknodata 

1 -nodata -10 -i geodata/vegetation/ETmean08-11.tif -o geodata/vegetation/ETmean08-11_01_msk.tif


CalledProcessError: Command 'b'pksetmask -co COMPRESS=DEFLATE -co ZLEVEL=9 \\\n-m geodata/vegetation/ETmean08-11_01_trhA.tif  -msknodata 1 -nodata  -9 \\\n-m geodata/vegetation/ETmean08-11_01_trhB.tif  -msknodata 1 -nodata  -5 \\\n-m geodata/vegetation/ETmean08-11_01_trhC.tif  -msknodata 1 -nodata -10 \\\n-i geodata/vegetation/ETmean08-11.tif -o geodata/vegetation/ETmean08-11_01_msk.tif\n'' returned non-zero exit status 134.

**pyjeo**

In pyjeo, we can apply the mask in a "pythonic" way using [set items](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html#set-jim-items)

In [6]:
jim[mask1] = -9
jim[mask2] = -5
jim[mask3] = -10

NameError: name 'jim' is not defined

However, we can even skip the intermediate step of creating the mask:

In [7]:
jim = pj.Jim(fn)

jim[(jim<0) | (jim>10)] = -10
jim[(jim>=5) & (jim<=8)] = -5
jim[(jim>=1) & (jim<=2)] = -9

NameError: name 'pj' is not defined

The result can then be written on disk if needed:

## Exercise: check if pktools and pyjeo results are equal

In [8]:
jim0 = pj.Jim('geodata/vegetation/ETmean08-11_01_msk.tif')
jim0.properties.isEqual(jim)

NameError: name 'pj' is not defined

In [9]:
jim.properties.isEqual(pj.Jim('geodata/vegetation/ETmean08-11_01_msk.tif'))

NameError: name 'jim' is not defined

In [10]:
jim.io.write('geodata/vegetation/ETmean08-11_01_msk_pyjeo.tif', co = ['COMPRESS=DEFLATE', 'ZLEVEL=9'])

NameError: name 'jim' is not defined

## Composite images

**pktools** 

Create a mask to apply during the composite

In [11]:
%%bash
pkgetmask  -co COMPRESS=DEFLATE -co ZLEVEL=9 -min 0 -max 25 -data 0 -nodata 1 -ot Byte -i geodata/LST/LST_MOYDmax_month1.tif -o geodata/LST/LST_MOYDmax_month1-msk.tif

0

terminate called after throwing an instance of '

std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >

'


bash: line 1: 164657 Aborted                 (core dumped) pkgetmask -co COMPRESS=DEFLATE -co ZLEVEL

=9 -min 0 -max 25 -data 0 -nodata 1 -ot Byte -i geodata/LST/LST_MOYDmax_month1.tif -o geodata/LST/LS

T_MOYDmax_month1-msk.tif


CalledProcessError: Command 'b'pkgetmask  -co COMPRESS=DEFLATE -co ZLEVEL=9 -min 0 -max 25 -data 0 -nodata 1 -ot Byte -i geodata/LST/LST_MOYDmax_month1.tif -o geodata/LST/LST_MOYDmax_month1-msk.tif\n'' returned non-zero exit status 134.

Calculate mean and standard deviation with several images with [pkcomposite](http://pktools.nongnu.org/html/md_pkcomposite.html)

In [12]:
%%bash
pkcomposite $(for file in geodata/LST/LST_MOYDmax_month??.tif geodata/LST/LST_MOYDmax_month?.tif; do echo -i $file; done) \
-m geodata/LST/LST_MOYDmax_month1-msk.tif -msknodata 0 -cr mean   -dstnodata 0 \
-co  COMPRESS=LZW -co ZLEVEL=9 -o geodata/LST/LST_MOYDmax_monthMean.tif

pkcomposite $(for file in geodata/LST/LST_MOYDmax_month?.tif geodata/LST/LST_MOYDmax_month??.tif; do echo -i $file; done) \
-m geodata/LST/LST_MOYDmax_month1-msk.tif -msknodata 0 -cr stdev   -dstnodata -1 \
-co  COMPRESS=LZW -co ZLEVEL=9 -o geodata/LST/LST_MOYDmax_monthStdev.tif

FileOpenError geodata/LST/LST_MOYDmax_month??.tif
FileOpenError geodata/LST/LST_MOYDmax_month?.tif


ERROR 1: Attempt to create -2147483648x-2147483648 dataset is illegal,sizes must be larger than zero

.
Warning

: could not write projection information in geodata/LST/LST_MOYDmax_monthMean.tif


bash: line 3: 164674 Segmentation fault      (core dumped) pkcomposite $(for file in geodata/LST/LST

...

bash: line 7: 164722 Segmentation fault      (core dumped) pkcomposite $(for file in geodata/LST/LST

_MOYDmax_month?.tif geodata/LST/LST_MOYDmax_month??.tif;
do
    echo -i $file;
done) -m geodata/LST/

LST_MOYDmax_month1-msk.tif -msknodata 0 -cr stdev -dstnodata -1 -co COMPRESS=LZW -co ZLEVEL=9 -o geo

data/LST/LST_MOYDmax_monthStdev.tif


CalledProcessError: Command 'b'pkcomposite $(for file in geodata/LST/LST_MOYDmax_month??.tif geodata/LST/LST_MOYDmax_month?.tif; do echo -i $file; done) \\\n-m geodata/LST/LST_MOYDmax_month1-msk.tif -msknodata 0 -cr mean   -dstnodata 0 \\\n-co  COMPRESS=LZW -co ZLEVEL=9 -o geodata/LST/LST_MOYDmax_monthMean.tif\n\npkcomposite $(for file in geodata/LST/LST_MOYDmax_month?.tif geodata/LST/LST_MOYDmax_month??.tif; do echo -i $file; done) \\\n-m geodata/LST/LST_MOYDmax_month1-msk.tif -msknodata 0 -cr stdev   -dstnodata -1 \\\n-co  COMPRESS=LZW -co ZLEVEL=9 -o geodata/LST/LST_MOYDmax_monthStdev.tif\n'' returned non-zero exit status 139.

An alternative way is to use [pkstatprofile](http://pktools.nongnu.org/html/pkstatprofile.html)

In [13]:
%%bash
# Create a multiband vrt
gdalbuildvrt -overwrite -separate geodata/LST/LST_MOYDmax_month.vrt geodata/LST/LST_MOYDmax_month?.tif geodata/LST/LST_MOYDmax_month??.tif
# Calculate mean and standard deviation
pkstatprofile -co  COMPRESS=LZW -nodata 0 -f mean -f stdev  -i geodata/LST/LST_MOYDmax_month.vrt -o geodata/LST/LST_MOYDmax_month_mean_stdev.tif

0...10...20...30...40...50

Warning 1: Can't open geodata/LST/LST_MOYDmax_month?.tif. Skipping it


Warning 1: Can't open geodata/LST/LST_MOYDmax_month??.tif. Skipping it


...60...70...80...90...100 - done.


FileOpenError


CalledProcessError: Command 'b'# Create a multiband vrt\ngdalbuildvrt -overwrite -separate geodata/LST/LST_MOYDmax_month.vrt geodata/LST/LST_MOYDmax_month?.tif geodata/LST/LST_MOYDmax_month??.tif\n# Calculate mean and standard deviation\npkstatprofile -co  COMPRESS=LZW -nodata 0 -f mean -f stdev  -i geodata/LST/LST_MOYDmax_month.vrt -o geodata/LST/LST_MOYDmax_month_mean_stdev.tif\n'' returned non-zero exit status 1.

**pyjeo**

In pyjeo, we can can composite images using [reducePlane](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html?highlight=reduceplane#geometry._Geometry.reducePlane)

First we create a Jim object for all monthly GeoTIFF files containing LST data

In [14]:
# iterate over files in
# that directory
from datetime import datetime
files1 = sorted(Path('geodata/LST/').glob('LST_MOYDmax_month?.tif'))
files2 = sorted(Path('geodata/LST/').glob('LST_MOYDmax_month??.tif'))

#create single band multi-plane image
mask = pj.Jim()
jim = pj.Jim()
for file in files1 + files2:
    month = file.name.replace('LST_MOYDmax_month','').replace('.tif','')
    date = datetime.strptime('2019-' + month + '-01','%Y-%m-%d')
    jim.geometry.stackPlane(pj.Jim(file))
    jim.properties.setDimension(date, 'plane', append = True)
jim.properties.setDimension(['LST'], 'band')

NameError: name 'pj' is not defined

In [15]:
jim.properties.getDimension()

NameError: name 'jim' is not defined

## Exercise  
use the Xarray representation of Jim to display details on the dimension and variables of the jim object

In [16]:
jim.xr()

NameError: name 'jim' is not defined

We then create a mask based on the first month (plane is 0) and calculate the mean composite

In [17]:
print("crop")
mask = pj.geometry.cropPlane(jim, 0)
print("mask")
mask = (mask >= 0) & (mask <= 25)
print("set mask")
jim[mask] = 0
print("reduce plane")
mean = pj.geometry.reducePlane(jim, rule = 'mean', nodata = 0)

crop


NameError: name 'pj' is not defined

## Exercise  
The reducePlane operation takes a long time to process. The reason is the nodata value that complicates the calculation considerably. Create a new composite `mean_alldata` that does take into account all pixel values (without considering nodata values). Notice how the execution time is reduced...

In [18]:
mean_alldata = pj.geometry.reducePlane(jim, rule = 'mean')

NameError: name 'pj' is not defined

**Bridging pyjeo to Numpy**

Pyjeo has been designed to allow for [bridging](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/2_tutorial.html#bridging-jim-to-third-party-packages) Jim raster image objects to third party libraries such as Numpy.

We will re-use the mask Jim object to store the results.

In [19]:
import numpy as np

mask.pixops.convert('GDT_Float32')
jim.np()[jim.np()==0] = np.nan

mask.np()[:] = np.nanmean(jim.np(), axis=0)
mask.geometry.stackBand(mask)
mask.np(1)[:] = np.nanstd(jim.np(), axis=0)

ModuleNotFoundError: No module named 'numpy'

We can avoid NaN in the resulting image by replacing it with 0

In [20]:
mask.np()[:] = np.nan_to_num(np.nanmean(jim.np(), axis=0), nan=0)
mask.np(1)[:] = np.nan_to_num(np.nanstd(jim.np(), axis=0), nan=0)

NameError: name 'np' is not defined

Even better is to avoid duplication of data to reduce memory footprint

In [21]:
mask.np()[:] = np.nan_to_num(np.nanmean(jim.np(), axis=0), copy = False, nan=0)
mask.geometry.stackBand(mask)
mask.np(1)[:] = np.nan_to_num(np.nanstd(jim.np(), axis=0), copy = False, nan=0)

NameError: name 'np' is not defined

In [22]:
plt.gray()  # show the filtered result in grayscale
fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(131)  # left side
ax2 = fig.add_subplot(132)  # middle
ax3 = fig.add_subplot(133)  # right side
ax1.imshow(jim.np()[0,:,:])
ax2.imshow(mask.np()[:,:])
ax3.imshow(mask.np(1)[:,:])
plt.show()

NameError: name 'plt' is not defined

**Bridging pyjeo to Xarray**

In [23]:
jim.xr()

NameError: name 'jim' is not defined

In [24]:
xrmean = jim.xr().where(jim.xr()!=0).mean(dim = 'time', skipna=True)

NameError: name 'jim' is not defined

In [25]:
xrstd = jim.xr().where(jim.xr()!=0).std(dim = 'time', skipna=True)

NameError: name 'jim' is not defined

In [26]:
jim.xr().isel(time = 0).LST.plot(cmap='gray')

NameError: name 'jim' is not defined

In [27]:
xrmean.LST.plot(cmap='gray')

NameError: name 'xrmean' is not defined

In [28]:
xrstd.LST.plot(cmap='gray')

NameError: name 'xrstd' is not defined

Xarray supports many operations on time series. For instance, we can alculate the seasonal median of LST.  
Hint: check this website on [Digital Earth Africa](https://docs.digitalearthafrica.org/en/latest/sandbox/notebooks/Frequently_used_code/Working_with_time.html)

Xarray has implemented convenient wrappers for plotting. Plot the 12 months of LST via Xarray representation of Jim

In [29]:
jim.xr().LST.plot(col='time', col_wrap=6)

NameError: name 'jim' is not defined

## Exercise
Xarray has implemented convenient wrappers for plotting. Plot the four seasons in four different colums (see same website on [Digital Earth Africa](https://docs.digitalearthafrica.org/en/latest/sandbox/notebooks/Frequently_used_code/Working_with_time.html)

In [30]:
ds_seasonal.LST.plot(col='season', col_wrap=4)

NameError: name 'ds_seasonal' is not defined

In [31]:
ds_seasonal

NameError: name 'ds_seasonal' is not defined

## Using tiling mechanism

In [32]:
tiletotal = 4
overlap = 0
def reduceTile(tileindex):
    jim = pj.Jim()
    for file in files1 + files2:
        print(file)
        month = file.name.replace('LST_MOYDmax_month','').replace('.tif','')
        date = datetime.strptime('2019-' + month + '-01','%Y-%m-%d')
        jim.geometry.stackPlane(pj.Jim(file, tileindex = tileindex, tiletotal = tiletotal, overlap = overlap))
        jim.properties.setDimension(date, 'plane', append = True)
    jim.properties.setDimension(['LST'], 'band')
    print("crop")
    mask = pj.geometry.cropPlane(jim, 0)
    print("mask")
    mask = (mask >= 0) & (mask <= 25)
    print("set mask")
    jim[mask] = 0
    return pj.geometry.reducePlane(jim, rule = 'mean', nodata = 0)

In [33]:
plt.gray()  # show the filtered result in grayscale
fig = plt.figure(figsize=(20,20))
ax = []
ax.append(fig.add_subplot(221))
ax.append(fig.add_subplot(222))
ax.append(fig.add_subplot(223))
ax.append(fig.add_subplot(224))

for tileindex in range(4):
    ax[tileindex].imshow(reduceTile(tileindex).np())
plt.show()

NameError: name 'plt' is not defined

## Filter images

**pktools** 

Aggregating and filtering images using [pkfilter](http://pktools.nongnu.org/html/pkfilter.html)

In [34]:
%%bash
# mean aggregation 
pkfilter -co COMPRESS=DEFLATE -co ZLEVEL=9 -nodata 0 -ot Float32 -dx 10 -dy 10 -f mean -d 10 -i geodata/LST/LST_MOYDmax_monthMean.tif -o geodata/LST/LST_MOYDmax_monthMean_aggreate10mean.tif
# mean circular moving window
pkfilter -co COMPRESS=DEFLATE -co ZLEVEL=9 -nodata 0 -ot Float32 -dx 11 -dy 11 -f mean -circ  -i geodata/LST/LST_MOYDmax_monthMean.tif -o geodata/LST/LST_MOYDmax_monthMean_circular11mean.tif

terminate called after throwing an instance of 'std::__cxx11::basic_string<char, std::char_traits<ch

ar>, std::allocator<char> >'


bash: line 2: 164761 Aborted                 (core dumped) pkfilter -co COMPRESS=DEFLATE -co ZLEVEL=

9 -nodata 0 -ot Float32 -dx 10 -dy 10 -f mean -d 10 -i geodata/LST/LST_MOYDmax_monthMean.tif -o geod

ata/LST/LST_MOYDmax_monthMean_aggreate10mean.tif


...

ar>, std::allocator<char> >'


bash: line 4: 164808 Aborted                 (core dumped) pkfilter -co COMPRESS=DEFLATE -co ZLEVEL=

9 -nodata 0 -ot Float32 -dx 11 -dy 11 -f mean -circ -i geodata/LST/LST_MOYDmax_monthMean.tif -o geod

ata/LST/LST_MOYDmax_monthMean_circular11mean.tif


CalledProcessError: Command 'b'# mean aggregation \npkfilter -co COMPRESS=DEFLATE -co ZLEVEL=9 -nodata 0 -ot Float32 -dx 10 -dy 10 -f mean -d 10 -i geodata/LST/LST_MOYDmax_monthMean.tif -o geodata/LST/LST_MOYDmax_monthMean_aggreate10mean.tif\n# mean circular moving window\npkfilter -co COMPRESS=DEFLATE -co ZLEVEL=9 -nodata 0 -ot Float32 -dx 11 -dy 11 -f mean -circ  -i geodata/LST/LST_MOYDmax_monthMean.tif -o geodata/LST/LST_MOYDmax_monthMean_circular11mean.tif\n'' returned non-zero exit status 134.

**pyjeo**

In pyjeo, we can can use [filter](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html?highlight=filter2d#ngbops._NgbOps.filter2d) method, but more functions are available via bridging pyjeo to third party libraries such as [scipy]()

In [35]:
import pyjeo as pj
from pathlib import Path
from scipy import ndimage
import numpy as np
fn = Path('geodata/LST/LST_MOYDmax_monthMean.tif')
jim = pj.Jim(fn)
taps = np.ones((10, 10))
mean = pj.ngbops.firfilter2d(jim, taps=taps, norm=True, pad='symmetric')
print(mean.properties.nrOfCol(), mean.properties.nrOfRow())
mean = mean[::10,::10]
print(mean.properties.nrOfCol(), mean.properties.nrOfRow())

ModuleNotFoundError: No module named 'pyjeo'

In [36]:
plt.gray()  # show the filtered result in grayscale
fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(121)  # left side
ax2 = fig.add_subplot(122)  # right side
ax1.imshow(jim.np())
ax2.imshow(mean.np())
plt.show()

NameError: name 'plt' is not defined

In [37]:
def unit_circle(r):
    A = np.arange(-r,r+1)**2
    dists = np.sqrt(A[:,None] + A)
    return (dists<=r).astype(int)
unit_circle(10)

NameError: name 'np' is not defined

In [38]:
mean = pj.ngbops.firfilter2d(jim, taps=unit_circle(10), norm=True, pad='symmetric')

NameError: name 'pj' is not defined

In [39]:
plt.gray()  # show the filtered result in grayscale
fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(121)  # left side
ax2 = fig.add_subplot(122)  # right side
ax1.imshow(jim.np())
ax2.imshow(mean.np())
plt.show()

NameError: name 'plt' is not defined

More functions are available via bridging pyjeo to third party libraries such as Multidimensional image processing from [scipy](https://docs.scipy.org/doc/scipy/tutorial/ndimage.html)

In [40]:
jim_filtered = pj.Jim(jim)
jim_filtered.np()[:] = ndimage.gaussian_filter(jim.np(), sigma = 2)
ndimage.gaussian_filter(jim.np(), sigma = 2, output=jim.np())
assert jim.properties.isEqual(jim_filtered)

NameError: name 'pj' is not defined

## Images statistics

**pktools** 

Aggregating and filtering images using [pkstat](http://pktools.nongnu.org/html/pkstat.html)

In [41]:
%%bash
pkstat -hist  -src_min 0  -i geodata/temperature/ug_bio_3.tif > geodata/temperature/ug_bio_3.hist
head geodata/temperature/ug_bio_3.hist

bash: line 1: geodata/temperature/ug_bio_3.hist: No such file or directory


head: cannot open 'geodata/temperature/ug_bio_3.hist' for reading

: No such file or directory


CalledProcessError: Command 'b'pkstat -hist  -src_min 0  -i geodata/temperature/ug_bio_3.tif > geodata/temperature/ug_bio_3.hist\nhead geodata/temperature/ug_bio_3.hist\n'' returned non-zero exit status 1.

In [42]:
%%bash
pkstat -hist  -nbin  20 -src_min 0  -i geodata/vegetation/GPPstdev08-11.tif

FileOpenError


**pyjeo**

In pyjeo, we can can use [getStats](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html?highlight=getstats#stats._Stats.getStats)

In [43]:
fn = Path('geodata/temperature/ug_bio_3.tif')
jim = pj.Jim(fn)
stats = jim.stats.getStats('histogram', src_min = 0)
print(stats['bin'][0:10])
print(stats['histogram'][0:10])

NameError: name 'pj' is not defined

In [44]:
fn = Path('geodata/vegetation/GPPstdev08-11.tif')
jim = pj.Jim(fn)
stats = jim.stats.getStats('histogram', src_min = 0, nbin = 20)
for index, bin in enumerate(stats['bin']):
    print(bin, stats['histogram'][index])

NameError: name 'pj' is not defined

In [45]:
fig = plt.figure(figsize = (10, 5))
plt.bar(stats['bin'],stats['histogram'])
plt.xlabel("pixel value")
plt.ylabel("abs frequency")
plt.title("Histogram of pixel values")
plt.show()

NameError: name 'plt' is not defined

## Images reclassification

**pktools** 

Aggregating and filtering images using [pkreclass](http://pktools.nongnu.org/html/pkreclass.html)

In [46]:
%%bash
pkstat -hist -i geodata/temperature/ug_bio_3.tif | grep -v " 0" | awk '{ if ($1<75) {print $1, 0} else {print $1 , 1}}' > geodata/temperature/reclass_ug_bio_3.txt
pkreclass -co COMPRESS=DEFLATE -co ZLEVEL=9 -code geodata/temperature/reclass_ug_bio_3.txt -i geodata/temperature/ug_bio_3.tif  -o geodata/temperature/reclass_ug_bio_3.tif

bash: line 1: geodata/temperature/reclass_ug_bio_3.txt: No such file or directory


terminate called after throwing an instance of 'std::__cxx11::basic_string<char, std::char_traits<ch

ar>, std::allocator<char> >'


bash: line 2: 164845 Aborted                 (core dumped) pkreclass -co COMPRESS=DEFLATE -co ZLEVEL

=9 -code geodata/temperature/reclass_ug_bio_3.txt -i geodata/temperature/ug_bio_3.tif -o geodata/tem

perature/reclass_ug_bio_3.tif


CalledProcessError: Command 'b'pkstat -hist -i geodata/temperature/ug_bio_3.tif | grep -v " 0" | awk \'{ if ($1<75) {print $1, 0} else {print $1 , 1}}\' > geodata/temperature/reclass_ug_bio_3.txt\npkreclass -co COMPRESS=DEFLATE -co ZLEVEL=9 -code geodata/temperature/reclass_ug_bio_3.txt -i geodata/temperature/ug_bio_3.tif  -o geodata/temperature/reclass_ug_bio_3.tif\n'' returned non-zero exit status 134.

**pyjeo**

In pyjeo, we can can use [reclass](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html?highlight=reclass#classify._Classify.reclass)

In [47]:
fn = 'geodata/temperature/ug_bio_3.tif'
jim = pj.Jim(fn)
print(jim.properties.getDataType())
print(jim.stats.getStats())
stats = jim.stats.getStats('histogram')

for index, bin in enumerate(stats['bin']):
    if stats['histogram'][index] > 0:
        print(bin, stats['histogram'][index])

if -9999 in jim.np():
    print("value -9999 is found")
classes0 = [c for c in stats['bin'] if stats['histogram'][stats['bin'].index(c)] > 0 and c < 75]
classes1 = [c for c in stats['bin'] if stats['histogram'][stats['bin'].index(c)] > 0 and c >= 75]
reclasses0 = np.zeros_like(classes0).tolist()
reclasses1 = np.ones_like(classes1).tolist()
print(classes0 + classes1)
print(reclasses0 + reclasses1)

reclass = pj.classify.reclass(jim, classes = classes0 + classes1, reclasses = reclasses0 + reclasses1)

NameError: name 'pj' is not defined

However, we can do it much simpler:

In [48]:
jim[jim < 75] = 0
jim[jim >= 75] = 1
print(reclass.properties.isEqual(jim))

NameError: name 'jim' is not defined

## Zonal statistic (polygon extraction)

**pktools** 

Aggregating and filtering images using [extractogr](http://pktools.nongnu.org/html/pkextractogr.html)

In [49]:
%%bash
rm -f geodata/shp/polygons_stat.*
pkextractogr -srcnodata -339999995214436424907732413799364296704   -r mean -r stdev -r min -i geodata/vegetation/GPPmean08-11.tif -s geodata/shp/polygons.sqlite -o geodata/shp/polygons_stat.sqlite
pkextractogr -f "ESRI Shapefile" -srcnodata -339999995214436424907732413799364296704   -r mean -r stdev -r min -i geodata/vegetation/GPPmean08-11.tif -s geodata/shp/polygons.sqlite -o geodata/shp/polygons_stat.shp

# we can also create a csv that can be manipulate later on with awk
rm  -f geodata/shp/polygons_stat.csv
pkextractogr -f CSV -srcnodata -339999995214436424907732413799364296704   -r mean -r stdev -r min -i geodata/vegetation/GPPmean08-11.tif -s geodata/shp/polygons.sqlite -o geodata/shp/polygons_stat.csv

FileOpenError


FileOpenError


FileOpenError


CalledProcessError: Command 'b'rm -f geodata/shp/polygons_stat.*\npkextractogr -srcnodata -339999995214436424907732413799364296704   -r mean -r stdev -r min -i geodata/vegetation/GPPmean08-11.tif -s geodata/shp/polygons.sqlite -o geodata/shp/polygons_stat.sqlite\npkextractogr -f "ESRI Shapefile" -srcnodata -339999995214436424907732413799364296704   -r mean -r stdev -r min -i geodata/vegetation/GPPmean08-11.tif -s geodata/shp/polygons.sqlite -o geodata/shp/polygons_stat.shp\n\n# we can also create a csv that can be manipulate later on with awk\nrm  -f geodata/shp/polygons_stat.csv\npkextractogr -f CSV -srcnodata -339999995214436424907732413799364296704   -r mean -r stdev -r min -i geodata/vegetation/GPPmean08-11.tif -s geodata/shp/polygons.sqlite -o geodata/shp/polygons_stat.csv\n'' returned non-zero exit status 1.

**Zonal statistic (point extraction)**

In [50]:
import functools
import time

def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer

In [51]:
%%bash 
# at point location
rm -f geodata/shp/point_stat.csv
pkextractogr -f CSV -srcnodata -339999995214436424907732413799364296704 -r mean -r stdev -r min -i geodata/vegetation/GPPmean08-11.tif -s geodata/shp/presence.shp -o geodata/shp/point_stat.csv
# at point location + 1 pixel around 
rm -f geodata/shp/point-buf_stat.csv
pkextractogr -f CSV -buf 2 -srcnodata -339999995214436424907732413799364296704 -r mean -r stdev -r min -i geodata/vegetation/GPPmean08-11.tif -s geodata/shp/presence.shp -o geodata/shp/point-buf_stat.csv

FileOpenError


FileOpenError


CalledProcessError: Command 'b'# at point location\nrm -f geodata/shp/point_stat.csv\npkextractogr -f CSV -srcnodata -339999995214436424907732413799364296704 -r mean -r stdev -r min -i geodata/vegetation/GPPmean08-11.tif -s geodata/shp/presence.shp -o geodata/shp/point_stat.csv\n# at point location + 1 pixel around \nrm -f geodata/shp/point-buf_stat.csv\npkextractogr -f CSV -buf 2 -srcnodata -339999995214436424907732413799364296704 -r mean -r stdev -r min -i geodata/vegetation/GPPmean08-11.tif -s geodata/shp/presence.shp -o geodata/shp/point-buf_stat.csv\n'' returned non-zero exit status 1.

**pyjeo**

In pyjeo, we can can use [extract](https://jeodpp.jrc.ec.europa.eu/services/processing/pyjeohelp/3_reference.html#geometry._GeometryVect.extract)

output in SQLite format

In [52]:
fn = 'geodata/vegetation/GPPmean08-11.tif'
vfn = 'geodata/shp/polygons.sqlite'
jim = pj.Jim(fn)
jim[jim<0]=-1
print(jim.stats.getStats())
v = pj.JimVect(vfn)
output = 'geodata/shp/temp.sqlite'
Path(output).unlink(missing_ok = True)
extracted1 = pj.geometry.extract(v, jim, output=output, rule=['mean', 'stdev', 'min'], srcnodata = -1)

NameError: name 'pj' is not defined

output in ESRI Shapefile format

In [53]:
output = 'geodata/shp/temp.shp'
Path(output).unlink(missing_ok = True)
extracted2 = pj.geometry.extract(v, jim, rule=['mean', 'stdev', 'min'], output='geodata/shp/temp.shp', oformat='ESRI Shapefile', srcnodata = -1)

NameError: name 'pj' is not defined

calculate in memory and get result in dictionary

In [54]:
extracted3 = pj.geometry.extract(v, jim, rule=['allpoints'], output='temp1', oformat='Memory', srcnodata = -1)

NameError: name 'pj' is not defined

In pandas format

In [55]:
pd.DataFrame(extracted3.dict())

NameError: name 'pd' is not defined

Extract point data

In [56]:
vfn = 'geodata/shp/presence.shp'
v = pj.JimVect(vfn)
extracted4 = pj.geometry.extract(v, jim, rule=['mean'], output='point', oformat='Memory', srcnodata = -1)

NameError: name 'pj' is not defined

Extract points with buffer to calculate mean and standard deviation and minimum

In [57]:
buffer = jim.properties.getDeltaX()*1
extracted5 = pj.geometry.extract(v, jim, rule=['mean', 'stdev', 'min'], output='point_buf', oformat='Memory', srcnodata = -1, buffer = buffer)
pd.DataFrame(extracted5.dict())

NameError: name 'jim' is not defined

**Remove all the output**

In [58]:
%%bash
rm -f  geodata/vegetation/GPPcv08-11.tif geodata/LST/*_crop.tif geodata/vegetation/ETmean08-11_crop_trh.tif geodata/vegetation/ETmean08-11_crop_trh.txt vegetation/ETmean08-11_crop.txt geodata/vegetation/ETmosaic.vrt geodata/vegetation/ETmosaic.tif geodata/vegetation/stack_??.tifgeodata/vegetation/stack.vrtgeodata/vegetation/tiles.*geodata/vegetation/ETmean08-11_crop_proximity.tifgeodata/vegetation/ETmean08-11_crop_buffer.tif geodata/dem/slope.tifgeodata/dem/aspect.tif geodata/dem/tri.tifgeodata/dem/tpi.tifgeodata/dem/roughness.tifgeodata/vegetation/ETmean08-11_01_trh?.tifgeodata/LST/LST_MOYDmax_month1-msk.tifgeodata/LST/LST_MOYDmax_monthStdev.tifgeodata/LST/LST_MOYDmax_monthMean.tifgeodata/LST/LST_MOYDmax_month_mean_stdev.tifgeodata/LST/LST_MOYDmax_month.vrtgeodata/LST/LST_MOYDmax_monthMean_aggreate10mean.tifgeodata/LST/LST_MOYDmax_monthMean_circular11mean.tif geodata/temperature/reclass_ug_bio_3.tifgeodata/temperature/reclass_ug_bio_3.txtgeodata/shp/polygons_stat.csvgeodata/shp/point-buf_stat.csvgeodata/shp/point_stat.csvgeodata/shp/polygons_stat.*geodata/shp/TM_LARGE_BORDERS.shp.* geodata/shp/TM_UGANDA_BORDERS-0.3.* geodata/vegetation/ETmean08-11_crop.txt geodata/vegetation/ETmean08-11_01_msk_pyjeo.tif geodata/shp/temp.*